In [45]:
import pandas as pd
import altair as alt

# Load the data
thefts = pd.read_parquet('../../data/clean/kia_hyundia_thefts.parquet')
city_lat_long = pd.read_csv('../../data/clean/city_lat_long.csv')
thefts = thefts.merge(city_lat_long, left_on='city', right_on='city', how='left')

thefts = thefts.dropna(subset=['percent_kia_hyundai'])


,date,city,count_kia_hyundai,count_all,percent_kia_hyundai,month,year,lat,long
0,2019-12-01,Denver,48.0,615.0,8.0,Dec,2019.0,39.7392,-104.9903
1,2020-01-01,Denver,21.0,519.0,4.0,Jan,2020.0,39.7392,-104.9903
2,2020-02-01,Denver,28.0,402.0,7.0,Feb,2020.0,39.7392,-104.9903
3,2020-03-01,Denver,35.0,508.0,7.0,Mar,2020.0,39.7392,-104.9903
4,2020-04-01,Denver,39.0,616.0,6.0,Apr,2020.0,39.7392,-104.9903


In [54]:
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, feature='states')

select_yearmonth = alt.selection_interval(encodings=['x'], name='brush')

area = alt.Chart(thefts).mark_area(color='#A56A6E').encode(
    x=alt.X('yearmonth(date):T', axis=alt.Axis(title='')),
    y=alt.Y('average(percent_kia_hyundai):Q', axis=alt.Axis(title='Average % of Thefts')),
    tooltip=['yearmonth(date):T', 'average(percent_kia_hyundai):Q']
).properties(
    width=500,
    height=100
).add_params(
    select_yearmonth
).properties(
    title=alt.Title(
        'Average Percent of Car Thefts that are Kia or Hyundai',
        subtitle='Drag to select a time range'
    )
)

background = alt.Chart(states).mark_geoshape(
    fill='white',
    stroke='#666666'
).properties(
    width=500,
    height=300
).project('albersUsa')

points = alt.Chart(thefts).mark_circle(
    size=20,
    color='#A56A6E',
    opacity=0.6
).encode(
    longitude='long:Q',
    latitude='lat:Q',
    tooltip=['city:N', 'percent_kia_hyundai:Q'],
    size=alt.condition(select_yearmonth, 'percent_kia_hyundai:Q', alt.value(0), title='Percent of Thefts')
).transform_filter(
    select_yearmonth
)

alt.vconcat(
    area,
    (background + points)
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=15,
    labelFont="Nunito Sans",
    titleFont="Nunito Sans"
).configure_legend(
    titleFontSize=15,
    labelFontSize=12,
    labelFont="Nunito Sans",
    titleFont="Nunito Sans",
    orient='bottom'
).configure_title(
    font='Nunito Sans',
    fontSize=22,
    subtitleFontSize=15,
    subtitleFont='Nunito Sans',
    subtitleColor='#666666'
)

alt.VConcatChart(...)